Per my understanding of the problem. to achieve result in a faster way we should not use xml or json style data. Flat text style file or CSV data would work faster than xml/json. Both in XML and JSON they are in multilevel format.

Here I had used JSON data of the same source data provided as XML, because for the purpose of using it through Spark environment using through python or pyspark I haven't found any suitable example leading to that. May be I haven't exhausted my search enough. It would be better to work with XML with scala. Lots of different examples have been provided. But there is third party tool a jar file prepared by Databricks to work XML files.

I had converted the xml files into json using a npm tool named xml2json. As I am using single machine so from spark point of view it is single core so for the sake of brevity I had picked up 5000 files out of 33000 files randomly.

I had tried working with the values instructed in the databricks site as mentioned below:
#import os
#os.environ['PYSPARK_SUBMIT_ARGS']='--packages com.databricks.spark.xml:spark-xml_2.11-0.5.0.jar pyspark-shell

I had downloaded that jar and placed it into the pyspark library location but somehow it did not respond as expected or I might have done some mistakes. I would be happier if somebody helps me to understand this part.

As I do have both Apache Spark and Pyspark so with the findspark utility of python I had searched for the spark environment. This is the extra tool I had used.

In [ ]:
# Finding appropriate spark environment and initializing it.
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SQLContext, SparkSession, functions as F
from pyspark.sql.functions import explode, col, expr, isnan, udf
from pyspark.sql.types import IntegerType, FloatType, StructType, StructField, StringType
import pyspark
import json
import datetime, time
from datetime import timedelta

# Creating the Spark Session and SQL Context
spark = SparkSession.builder.master("local").getOrCreate()
sqlContext = pyspark.SQLContext(spark)


#reading the JSON content from the path where I put all the json data.
path="/Users/rivthebest/Documents/INTERVIEW/trades-json-fraction"

In [5]:
# Creating the RDD/Data Frame
loadJSONDF1 = sqlContext.read.json(path, multiLine=True)

In [6]:
#Repartitioning the Data Frame
loanJSONDF1 = loadJSONDF1.repartition(1000)

In [7]:
#Printing the Schema
loanJSONDF1.printSchema()

root
 |-- loan: struct (nullable = true)
 |    |-- seller: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |-- trade: struct (nullable = true)
 |    |    |-- advance: struct (nullable = true)
 |    |    |    |-- amount: string (nullable = true)
 |    |    |    |-- amount_gbp: string (nullable = true)
 |    |    |    |-- date: string (nullable = true)
 |    |    |    |-- percentage: string (nullable = true)
 |    |    |-- arrears: struct (nullable = true)
 |    |    |    |-- in_arrears: string (nullable = true)
 |    |    |    |-- in_arrears_on_date: string (nullable = true)
 |    |    |-- crystalised_loss: struct (nullable = true)
 |    |    |    |-- amount: string (nullable = true)
 |    |    |    |-- date: string (nullable = true)
 |    |    |-- currency: string (nullable = true)
 |    |    |-- discount: struct (nullable = true)
 |    |    |    |-- on: string (nullable = true)
 |    |    |    |-- percentage: string (nullable = true)
 |    |    |-- expected_

In [8]:
#Printing First 5 JSON file Records
loanJSONDF1.take(5)

[Row(loan=Row(seller=Row(id='398'), trade=Row(advance=Row(amount='20927.0', amount_gbp='20927.0', date='2014-10-31', percentage='85.0'), arrears=Row(in_arrears='Yes', in_arrears_on_date='2014-11-26'), crystalised_loss=Row(amount='Yes', date='2014-12-08'), currency='GBP', discount=Row(on='Facevalue', percentage='0.8'), expected_payment_date='2014-12-12 00:00:00 +0000', face_value=Row(amount='24620.0', amount_gbp='24620.0'), gross_yield=Row(annualised='11.88184119551574'), id='10486', outstanding_principal=Row(amount='0.0', amount_gbp='0.0'), payment=Row(state='Repurchased'), price_grade='5', settlement_date='2014-12-08', type='MultiDebtor'))),
 Row(loan=Row(seller=Row(id='1010'), trade=Row(advance=Row(amount='19134.0', amount_gbp='13357.4454', date='2015-07-16', percentage='90.0'), arrears=Row(in_arrears='No', in_arrears_on_date='nan'), crystalised_loss=Row(amount='nan', date='nan'), currency='EUR', discount=Row(on='Advance', percentage='1.0'), expected_payment_date='2015-08-28 00:00:00

In [9]:
# A compact of View of Question 1
# to show more rows we need to give numerical amount in the show function as .show(<number of rows>)
# By default it will return 20 rows
loanJSONDF1.select(col("loan.seller.id").alias("Seller ID"), 
                      col("loan.trade.id").alias("Trade ID"),                                         
                      col("loan.trade.advance.date").alias("Trade Advance Date"),                                                              
                      col("loan.trade.expected_payment_date").alias("Expected Payment Date"),                                        
                      col("loan.trade.settlement_date").alias("Settlement Date"),
                      col("loan.trade.type").alias("Trade Type")) \
              .where(col("loan.trade.type").isin("Standard")).show()

+---------+--------+------------------+---------------------+---------------+----------+
|Seller ID|Trade ID|Trade Advance Date|Expected Payment Date|Settlement Date|Trade Type|
+---------+--------+------------------+---------------------+---------------+----------+
|     8398|   10422|        2014-10-30| 2014-11-30 00:00:...|     2015-01-20|  Standard|
|    10102|   15515|        2015-06-25| 2015-07-31 00:00:...|     2015-07-10|  Standard|
|      216|   17788|        2015-09-22| 2015-10-20 00:00:...|     2015-10-19|  Standard|
|      173|   11026|        2014-12-10| 2015-01-09 00:00:...|     2015-01-20|  Standard|
|     1291|   10746|        2014-11-20| 2014-12-18 00:00:...|     2014-12-01|  Standard|
|     1837|   15132|        2015-06-10| 2015-09-11 00:00:...|     2015-09-29|  Standard|
|    12853|   14529|        2015-05-15| 2015-07-04 00:00:...|     2016-03-21|  Standard|
|     4727|   13046|        2015-03-04| 2015-04-17 00:00:...|     2015-05-06|  Standard|
|     8887|   10234| 

In [ ]:
#Query with all attributes
#For Query 1

loanJSONDF1.select(col("loan.seller.id").alias("Seller ID"), 
                      col("loan.trade.id").alias("Trade ID"),
                      col("loan.trade.advance.amount").alias("Advance Amount"),
                      col("loan.trade.advance.amount_gbp").alias("Amount in GBP"),                     
                      col("loan.trade.advance.date").alias("Trade Advance Date"),
                      col("loan.trade.advance.percentage").alias("Advance Percentage"),
                      col("loan.trade.arrears.in_arrears").alias("Arrears"),
                      col("loan.trade.arrears.in_arrears_on_date").alias("Arrears On Date"),
                      col("loan.trade.crystalised_loss.amount").alias("Crystalised Loss"),
                      col("loan.trade.crystalised_loss.date").alias("Crystalised Loss Date"),
                      col("loan.trade.currency").alias("Currency"),
                      col("loan.trade.discount.on").alias("Discount On"),
                      col("loan.trade.discount.percentage").alias("Discount Percentage"),                      
                      col("loan.trade.expected_payment_date").alias("Expected Payment Date"),                                  
                      col("loan.trade.face_value.amount").alias("Face Value Amount"),
                      col("loan.trade.face_value.amount_gbp").alias("Face Value Amount GBP"),
                      col("loan.trade.gross_yield.annualised").alias("Groos Yield Annualised"),
                      col("loan.trade.outstanding_principal.amount").alias("Outstanding Principal Amount"),
                      col("loan.trade.outstanding_principal.amount_gbp").alias("Amount in GBP"),
                      col("loan.trade.payment.state").alias("Payment State"),
                      col("loan.trade.price_grade").alias("Price Grade"),
                      col("loan.trade.settlement_date").alias("Settlement Date"),
                      col("loan.trade.type").alias("Trade Type")) \
                 .where(col("loan.trade.type").isin("Standard")).show()

In [24]:
## For 2(a) Query

# There is nifty python code I had found in a blog providing here with for dealing with the schema 
    # evolution everytime required for running UDF/UDAF. This python code mentioned in the below blog
    # is a replacement of repeated botheration schema evolution
    
    ## http://nadbordrozd.github.io/blog/2016/05/22/one-weird-trick-that-will-fix-your-pyspark-schemas/

## Creating the new Schema
def build_schema():
    #updating the schema with new entry duration as loan.trade.duration
    # The schema has been placed inside a function
       
    newLoanSchema = StructType([
                    StructField("loan", 
                        StructType([
                            StructField("seller", 
                                StructType([
                                        StructField('id', StringType(), True)
                                         ]), 
                                    True),
                            StructField("trade", 
                                StructType([
                                    StructField("advance",
                                           StructType([
                                               StructField("amount", StringType(), True),
                                               StructField("amount_gbp", StringType(), True),
                                               StructField("date", StringType(), True),
                                               StructField("percentage", StringType(), True)
                                               ]),  True),
                                    StructField("arrears", 
                                            StructType([
                                                StructField("in_arrears", StringType(), True),
                                                StructField("in_arrears_on_date", StringType(), True)
                                               ]), True),
                                    StructField("crystalised_loss", 
                                            StructType([
                                                StructField("amount", StringType(), True),
                                                StructField("date", StringType(), True)
                                               ]), True),
                                    StructField("currency", StringType(), True),
                                    StructField("discount",
                                            StructType([
                                                StructField("on", StringType(), True),
                                                StructField("percentage", StringType(), True)
                                               ]), True),
                                    StructField("expected_payment_date", StringType(), True),
                                    StructField("duration", FloatType(), True),
                                    StructField("face_value", 
                                            StructType([
                                                StructField("amount", StringType(), True),
                                                StructField("gbp", StringType(), True)
                                               ]), True),
                                    StructField("gross_yield", 
                                            StructType([
                                                StructField("annualised", StringType(), True)
                                               ]), True),
                                    StructField("id", StringType(), True),
                                    StructField("outstanding_principal", 
                                            StructType([
                                                StructField("amount", StringType(), True),
                                                StructField("amount_gbp", StringType(), True)
                                               ]), True),
                                    StructField("payment", 
                                            StructType([
                                                StructField("state", StringType(), True)
                                               ]), True),
                                    StructField("price_grade", StringType(), True),
                                    StructField("settlement_date", StringType(), True),
                                    StructField("type", StringType(), True)
                                          ]), True)
                                  ]), True)
                    ])
    return newLoanSchema

In [27]:
#This is the difficult part where I get stuck, I am not sure where I am making mistakes
# The error message is also provided for your reference.
loadJSONDF2 = sqlContext.createDataFrame(loadJSONDF1.rdd, schema=build_schema())

AttributeError: 'SparkSession' object has no attribute 'serializer'

In [ ]:
#These are all runnable codes. If the code above line is fixed. This section would be running smoothly.

#Time duration calculation

def time_duration(y, x):
    y = y.split(' ')[0]
    end = datetime.datetime.strptime(y, "%Y-%m-%d")
    start = datetime.datetime.strptime(x, "%Y-%m-%d")
    delta = (end - start).total_seconds()
    delta = round((delta / (24 * 3600)))
    

# registering as udf
ftd = udf(time_duration,newLoanSchema)

# For the sake of operating the below code we might require to change the DataFrame name or may not

newRDD = loanJSONDF2.withColumn("loan.trade.duration",ftd(loanJSONDF2['loan.trade.expected_payment_date'],
                                         loanJSONDF2['loan.trade.advance.date']))

#printing the New Schema Again
#loanJSONRDDNew.printSchema()
newRDD.printSchema()

In [ ]:
# query with a duration as output in a compact view
loanJSONDF2.select(col("loan.seller.id").alias("Seller ID"), 
                      col("loan.trade.id").alias("Trade ID"),                                         
                      col("loan.trade.advance.date").alias("Trade Advance Date"),                                                              
                      col("loan.trade.expected_payment_date").alias("Expected Payment Date"),                   
                      col("loan.trade.duration").alias("Duration in Days"),                     
                      col("loan.trade.settlement_date").alias("Settlement Date"),
                      col("loan.trade.type").alias("Trade Type")) \
              .where(col("loan.trade.type").isin("Standard")).show()

In [ ]:
# testing date difference calculation
import time, datetime
from datetime import timedelta
y="2011-08-31 00:00:00 +0000"
x="2011-07-31"

y = y.split(' ')[0]
end = datetime.datetime.strptime(y, "%Y-%m-%d")
start = datetime.datetime.strptime(x, "%Y-%m-%d")
delta = (end - start).total_seconds() / (24 * 3600)
delta = round(delta)

print(delta)

In [28]:
#For 2b Query
# I haven't work on this query for finding the aggregate details.
loanJSONDF1.groupBy(['loan.seller.id']).agg(F.collect_list('loan.trade.id')).show()

+-----+---------------------------+
|   id|collect_list(loan.trade.id)|
+-----+---------------------------+
|13192|                    [14213]|
|14369|       [17661, 14768, 14...|
|15269|                    [15530]|
|15448|       [16324, 16331, 17...|
|15052|                    [16062]|
|13187|                    [13616]|
|16549|                    [17775]|
|11251|       [13188, 12968, 18...|
|  919|       [11393, 10987, 15...|
|12750|       [14836, 13331, 13...|
| 2550|       [17384, 16398, 13...|
| 9196|       [15028, 15207, 14...|
|17128|                    [17053]|
|  124|                      [150]|
|10805|       [13097, 17402, 11...|
|10481|       [11170, 17844, 15...|
| 8279|       [13105, 14140, 10...|
| 8744|       [13173, 10965, 13...|
|10668|       [13318, 11045, 13...|
| 2495|       [16330, 14987, 10...|
+-----+---------------------------+
only showing top 20 rows

